In [ ]:
# !pip install deep-translator

In [ ]:
# thai japan china german
from deep_translator import GoogleTranslator
import pandas as pd

def back_translate(text, src='en', mid='ja'):
    try:
        print(f"Original: {text}")
        translated = GoogleTranslator(source=src, target=mid).translate(text)
        print(f"Translated to Japanese: {translated}")
        back_translated = GoogleTranslator(source=mid, target=src).translate(translated)
        print(f"Back Translated to English: {back_translated}\n")  
        return back_translated
    except Exception as e:
        print(f"Error translating: {e}")
        return text  

# โหลดข้อมูล
print("Loading dataset...")
df = pd.read_csv('cleaned_dataset.csv')
print(f"Dataset loaded: {df.shape[0]} rows\n")

# เลือกเฉพาะ class ที่มีน้อย (mild, moderate, severe) มาเพิ่มข้อมูล
print("Filtering underrepresented classes (mild, moderate, severe)...")
df_aug = df[df['label'].isin(['mild', 'moderate', 'severe'])].copy()
print(f"Selected {df_aug.shape[0]} rows for augmentation\n")

# ใช้ Back Translation
print("Applying Back Translation...")
df_aug['text'] = df_aug['text'].apply(lambda x: back_translate(x))

# รวมข้อมูลใหม่กับข้อมูลเดิม
df = pd.concat([df, df_aug])
print(f"New dataset size after augmentation: {df.shape[0]} rows\n")

# บันทึกข้อมูลที่เพิ่มแล้ว
output_file = 'dataset_augmented01.csv'
df.to_csv(output_file, index=False)
print(f"Augmented dataset saved as {output_file}")


In [ ]:
import re

# โหลดข้อมูลจากไฟล์ที่คุณมี
file_path = "dataset_augmented01.csv"  
df = pd.read_csv(file_path)

# ตรวจสอบข้อมูลเบื้องต้น
print("ข้อมูลก่อนการทำความสะอาด:")
print(df.head())
print(df.info())

# ลบแถวที่มีค่าว่าง
df.dropna(inplace=True)

# ฟังก์ชันทำความสะอาดข้อความ
def clean_text(text):
    text = text.lower()  
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  
    text = re.sub(r'@\w+', '', text)  
    text = re.sub(r'[^a-z\s]', '', text)
    return text

# ใช้ฟังก์ชันทำความสะอาดกับคอลัมน์ "text"
df['text'] = df['text'].astype(str).apply(clean_text)
df.drop_duplicates(inplace=True)


# บันทึกไฟล์ใหม่หลังการทำความสะอาด
cleaned_file_path = "cleaned_dataset_augmented01.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"\nข้อมูลถูกทำความสะอาดแล้วและบันทึกเป็นไฟล์ {cleaned_file_path}")
print(df.head())


In [ ]:
# โหลดข้อมูลที่ทำความสะอาดแล้ว
cleaned_file_path = "cleaned_dataset_augmented01.csv"
df_cleaned = pd.read_csv(cleaned_file_path)

# ตรวจสอบข้อมูลหลังการทำความสะอาด
print("ข้อมูลหลังการทำความสะอาด:")
print(df_cleaned.head()) 
print(df_cleaned.info())  
print("\nตรวจสอบค่าที่หายไป:")
print(df_cleaned.isnull().sum())  

# ตรวจสอบค่าซ้ำซ้อน
print("\nตรวจสอบค่าซ้ำซ้อน:")
print(df_cleaned.duplicated().sum())  


In [ ]:
from deep_translator import GoogleTranslator
import pandas as pd

def back_translate(text, src='en', mid1='zh-CN', mid2='de'):
    try:
        print(f"Original: {text}") 
        translated1 = GoogleTranslator(source=src, target=mid1).translate(text)  
        print(f"Translated to Chinese: {translated1}")
        translated2 = GoogleTranslator(source=mid1, target=mid2).translate(translated1)  
        print(f"Translated to German: {translated2}")
        back_translated = GoogleTranslator(source=mid2, target=src).translate(translated2) 
        print(f"Back Translated to English: {back_translated}\n")
        return back_translated
    except Exception as e:
        print(f"Error translating: {e}")
        return text  

# โหลดข้อมูล
print("Loading dataset...")
df = pd.read_csv('cleaned_dataset_augmented01.csv')
print(f"Dataset loaded: {df.shape[0]} rows\n")

# นับจำนวนตัวอย่างในแต่ละ class
desired_count = 2500
label_counts = df['label'].value_counts()
print("Current label distribution:")
print(label_counts, "\n")

# เลือกเฉพาะ class ที่มีน้อยกว่า 2000
underrepresented_classes = label_counts[label_counts < desired_count].index.tolist()
print(f"Underrepresented classes: {underrepresented_classes}\n")

# เพิ่มข้อมูลสำหรับแต่ละ class
augmented_data = []
for label in underrepresented_classes:
    df_class = df[df['label'] == label].copy()
    num_needed = desired_count - len(df_class)
    print(f"Augmenting class '{label}' with {num_needed} more samples...")

    while len(augmented_data) < num_needed:
        for _, row in df_class.iterrows():
            new_text = back_translate(row['text'])
            augmented_data.append({'text': new_text, 'label': label})
            if len(augmented_data) >= num_needed:
                break

# สร้าง DataFrame ใหม่จากข้อมูลที่เพิ่ม
print("Creating augmented DataFrame...")
df_aug = pd.DataFrame(augmented_data)
print(f"Generated {df_aug.shape[0]} new rows.\n")

# รวมข้อมูลใหม่กับข้อมูลเดิม
df = pd.concat([df, df_aug], ignore_index=True)
print("New label distribution after augmentation:")
print(df['label'].value_counts(), "\n")

# บันทึกข้อมูลที่เพิ่มแล้ว
output_file = 'dataset_augmented02.csv'
df.to_csv(output_file, index=False)
print(f"Augmented dataset saved as {output_file}")


In [ ]:
import pandas as pd
import re

# โหลดข้อมูลจากไฟล์ที่คุณมี
file_path = "dataset_augmented02.csv" 
df = pd.read_csv(file_path)

# ตรวจสอบข้อมูลเบื้องต้น
print("ข้อมูลก่อนการทำความสะอาด:")
print(df.head())
print(df.info())

# ลบแถวที่มีค่าว่าง
df.dropna(inplace=True)

# ฟังก์ชันทำความสะอาดข้อความ
def clean_text(text):
    text = text.lower()  
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) 
    text = re.sub(r'@\w+', '', text) 
    text = re.sub(r'[^a-z\s]', '', text)  
    return text

# ใช้ฟังก์ชันทำความสะอาดกับคอลัมน์ "text"
df['text'] = df['text'].astype(str).apply(clean_text)
print("\nตรวจสอบค่าซ้ำซ้อน:")
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)


# บันทึกไฟล์ใหม่หลังการทำความสะอาด
cleaned_file_path = "cleaned_dataset_augmented02.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"\nข้อมูลถูกทำความสะอาดแล้วและบันทึกเป็นไฟล์ {cleaned_file_path}")
print(df.head())


In [ ]:
import pandas as pd

# โหลดข้อมูลที่ทำความสะอาดแล้ว
cleaned_file_path = "cleaned_dataset_augmented02.csv"
df_cleaned = pd.read_csv(cleaned_file_path)

# ตรวจสอบข้อมูลหลังการทำความสะอาด
print("ข้อมูลหลังการทำความสะอาด:")
print(df_cleaned.head())  #
print(df_cleaned.info())  

print("\nตรวจสอบค่าที่หายไป:")
print(df_cleaned.isnull().sum())  

# ตรวจสอบค่าซ้ำซ้อน
print("\nตรวจสอบค่าซ้ำซ้อน:")
print(df_cleaned.duplicated().sum()) 